# How Robust are Homographies?

To test robustness, let's check how robust against
- random noise
- outliers

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import scienceplots
import pandas as pd
from itertools import combinations
import pprint
from shapely.geometry import Polygon
plt.style.use(['science', 'grid', 'ieee'])

from utils import (
    load_dataset,
    get_images,
    transform_points,
    get_densities,
    predict,
    calculate_errors,
    get_result_dict,
    SCALING_FACTOR,
    plot_setup, 
    plot_setup_noised,
    plot_predictions,
    add_outlier
)

def filter_points(points, keys):
    filtered = {key: points[key] for key in keys}
    return filtered

def conduct_experiment(pv_img, tv_img, plot=True, print_errors=True, used_ref_points=None, return_homography=False):
    # Load Data
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)
    
    if used_ref_points:
        reference_pts_pv = filter_points(reference_pts_pv, used_ref_points)
        reference_pts_tv = filter_points(reference_pts_tv, used_ref_points)
        
    # Transform Points to Homogeneous Numpy arrays
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)
    # Calculate Homography
    h, _ = cv2.findHomography(
        reference_pts_pv_arr,
        reference_pts_tv_arr,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_inv = np.linalg.inv(h)

    # Get Pixel Densities
    reference_densities = get_densities(reference_pts_tv_arr, h_inv)
    validation_densities = get_densities(validation_pts_tv_arr, h_inv)
    
    # Predict Points
    predicted_reference_pts_tv = predict(reference_pts_pv_arr, h)
    predicted_validation_pts_tv = predict(validation_pts_pv_arr, h)
    
    # Errors
    reference_errors = calculate_errors(predicted_reference_pts_tv, reference_pts_tv_arr)
    validation_errors = calculate_errors(predicted_validation_pts_tv, validation_pts_tv_arr)

    # Combine Results in Dictionary
    reference_result_dict = get_result_dict(reference_pts_pv, reference_pts_tv_arr, predicted_reference_pts_tv, reference_errors, reference_densities, reference_pts_pv_arr)
    validation_result_dict = get_result_dict(validation_pts_pv, validation_pts_tv_arr, predicted_validation_pts_tv, validation_errors, validation_densities, reference_pts_pv_arr)
    
    # Print Errors
    if print_errors:
        print(f'Reference  Error: {np.mean(reference_errors) :.2f} PX!')
        print(f'Reference  Error: {np.mean(reference_errors) / SCALING_FACTOR * 100 :.2f} CM!')
        print('---')
        print(f'Validation Error: {np.mean(validation_errors):.2f} PX!')
        print(f'Validation Error: {np.mean(validation_errors) / SCALING_FACTOR * 100 :.2f} CM!')
    
    # Plots
    if plot:
        plot_setup(img_tv, img_pv, reference_result_dict, validation_result_dict)
        plot_predictions(img_tv, img_pv, reference_result_dict, validation_result_dict)
    if return_homography:
        return reference_result_dict, validation_result_dict, h
    return reference_result_dict, validation_result_dict

def get_collinearity_score(subset, reference_pts_pv):
    A = np.array(reference_pts_pv[subset[0]])
    B = np.array(reference_pts_pv[subset[1]])
    C = np.array(reference_pts_pv[subset[2]])
    AB = np.linalg.norm(A-B) 
    AC = np.linalg.norm(A-C) 
    BC = np.linalg.norm(B-C)
    distances = sorted([AB, AC, BC])
    return (distances[0] + distances[1] - distances[2]) / distances[2] * 100
    
def get_overall_collinearity_score(used_ref_pts, reference_pts_pv):
    scores = []
    for subset in combinations(used_ref_pts, 3):
        score = get_collinearity_score(subset, reference_pts_pv)
        scores.append(score)
    return min(scores)

def get_collinearity_scores(used_ref_pts, reference_pts_pv):
    scores = []
    for subset in combinations(used_ref_pts, 3):
        score = get_collinearity_score(subset, reference_pts_pv)
        scores.append(score)
    return scores

## Random Noise

Let's take our reference points and add random noise (gaussian) around it and conduct the experiments. We can make the same experiments as in the overall case, with all k and validation points.

We can conduct like 10 times random and one time the normal and then see how the result differs.

In [ ]:
def add_noise(points, m=0, std=1):
    zeros = np.zeros((points.shape[0], 1))
    noise = np.random.normal(m, std, points[:,:-1].shape)
    noise = np.hstack((noise, zeros))
    return points + noise


def get_result_dict_noised(
    point_dict_pv, 
    point_arr_tv, 
    predicted_points_tv, 
    errors, 
    densities, 
    point_arr_pv_noised, 
    point_arr_tv_noised
):
    error_dict = {}
    for i, (name, coord) in enumerate(point_dict_pv.items()):
        error_dict[name] = {}
        error_dict[name]['coordinates_pv'] = coord
        error_dict[name]['coordinates_tv'] = (point_arr_tv[i][0], point_arr_tv[i][1])
        error_dict[name]['coordinates_pv_noised'] = (point_arr_pv_noised[i][0], point_arr_pv_noised[i][1])
        error_dict[name]['coordinates_tv_noised'] = (point_arr_tv_noised[i][0], point_arr_tv_noised[i][1])
        error_dict[name]['predicted_coordinates_tv'] = (predicted_points_tv[i][0], predicted_points_tv[i][1])
        error_dict[name]['error'] = errors[i]
        error_dict[name]['pixel_density'] = densities[i]
    return error_dict

def conduct_experiment_noised(pv_img, tv_img, basic_homography, plot=True, print_errors=True, used_ref_points=None, n_mean=0, n_std=100):
    # Load Data
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)
    
    if used_ref_points:
        reference_pts_pv = filter_points(reference_pts_pv, used_ref_points)
        reference_pts_tv = filter_points(reference_pts_tv, used_ref_points)
        
    # Transform Points to Homogeneous Numpy arrays
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)

    # Add Noise
    reference_pts_pv_arr_noised = add_noise(reference_pts_pv_arr, n_mean, n_std)
    reference_pts_tv_arr_noised = add_noise(reference_pts_tv_arr, n_mean, n_std)
    
    # Calculate Homography
    h, _ = cv2.findHomography(
        reference_pts_pv_arr_noised,
        reference_pts_tv_arr_noised,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_inv = np.linalg.inv(h)

    # Get Pixel Densities
    reference_densities = get_densities(reference_pts_tv_arr, h_inv)
    # The densities should be calculated "perfectly" for the validation point, 
    # hence we should use a good homography for this, 
    # hence, we choose the baseline homography
    validation_densities = get_densities(validation_pts_tv_arr, np.linalg.inv(basic_homography)) # important trick!
    
    # Predict Points
    predicted_reference_pts_tv = predict(reference_pts_pv_arr_noised, h)
    predicted_validation_pts_tv = predict(validation_pts_pv_arr, h)
    
    # Errors
    reference_errors = calculate_errors(predicted_reference_pts_tv, reference_pts_tv_arr)
    validation_errors = calculate_errors(predicted_validation_pts_tv, validation_pts_tv_arr)

    # Combine Results in Dictionary
    reference_result_dict = get_result_dict_noised(
        reference_pts_pv, 
        reference_pts_tv_arr, 
        predicted_reference_pts_tv, 
        reference_errors, 
        reference_densities, 
        reference_pts_pv_arr_noised, 
        reference_pts_tv_arr_noised
    )
    validation_result_dict = get_result_dict(validation_pts_pv, validation_pts_tv_arr, predicted_validation_pts_tv, validation_errors, validation_densities, reference_pts_pv_arr)
     
    # Print Errors
    if print_errors:
        print(f'Reference  Error: {np.mean(reference_errors) :.2f} PX!')
        print(f'Reference  Error: {np.mean(reference_errors) / SCALING_FACTOR * 100 :.2f} CM!')
        print('---')
        print(f'Validation Error: {np.mean(validation_errors):.2f} PX!')
        print(f'Validation Error: {np.mean(validation_errors) / SCALING_FACTOR * 100 :.2f} CM!')
     
    # Plots
    if plot:
        plot_setup_noised(img_tv, img_pv, reference_result_dict, validation_result_dict)
        # plt.savefig('random_noise.png', dpi=300)
        plot_predictions(img_tv, img_pv, reference_result_dict, validation_result_dict)
    return reference_result_dict, validation_result_dict 

In [ ]:
baseline_val, baseline_ref, baseline_h = conduct_experiment('IMG_01', 'IMG_00', plot=False, return_homography=True)

In [ ]:
_ = conduct_experiment_noised('IMG_01', 'IMG_00', baseline_h, plot=False, n_std=50) 

In [ ]:
def noised_experiment(pv_img, tv_img, n_mean, n_stds, baseline_h):
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    rows = []

    for n_std in n_stds:
        print('-'*20, n_mean, n_std,'-'*20)
        for i in range(100):
            print('#'*10)
            print(f'Experiment {i}')
            reference_result_dict, validation_result_dict = conduct_experiment_noised(pv_img, 'IMG_00', baseline_h, plot=False, n_mean=n_mean, n_std=n_std)  
    
            for pt, pt_dict in validation_result_dict.items():
                pt_dict['experiment'] = i
                pt_dict['name'] = pt
                pt_dict['n_mean'] = n_mean
                pt_dict['n_std'] = n_std
                rows.append(pt_dict)
    df = pd.DataFrame(rows)
    df['img'] = pv_img
    return df
noise_results_0026 = noised_experiment('IMG_01', 'IMG_00', 0, [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70], baseline_h)
# noise_results_0029 = noised_experiment('IMG_02', 'IMG_00', 0, [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70], baseline_h)
# noise_results_0045 = noised_experiment('IMG_06', 'IMG_00', 0, [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70], baseline_h)

In [ ]:
# TODO ADD 0!!!
fig, ax = plt.subplots(1, 1)#, figsize=(10, 10))

grouped = noise_results_0026.groupby('n_std').agg(
    error_mean=('error', lambda x: np.mean(x).round(2)),
    error_std=('error', lambda x: np.std(x).round(2))
).reset_index()

means = grouped['error_mean'].values
stds = grouped['error_std'].values
noise = grouped['n_std'].values

means = np.insert(means, 0, 3.8, axis=0)
stds = np.insert(stds, 0, 0, axis=0)
noise = np.insert(noise, 0, 0, axis=0)

ax.plot(noise, means, c='black', marker='o')
ax.fill_between(noise, means-stds,means+stds,alpha=.1, color='black')
ax.set_xlabel('noise [px]')
ax.set_ylabel('error [px]')
ax.grid(False)

plt.savefig('output/noise_results.png', dpi=600)

In [ ]:
reference_result_dict, validation_result_dict = conduct_experiment('IMG_06', 'IMG_00', plot=False)  

In [ ]:
# TODO ADD 0!!!
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

grouped = noise_results_0045.groupby('n_std').agg(
    error_mean=('error', lambda x: np.mean(x).round(2)),
    error_std=('error', lambda x: np.std(x).round(2))
).reset_index()

means = grouped['error_mean'].values
stds = grouped['error_std'].values
noise = grouped['n_std'].values

# Add zero experiment
means = np.insert(means, 0, 34.93, axis=0)
stds = np.insert(stds, 0, 0, axis=0)
noise = np.insert(noise, 0, 0, axis=0)

ax.plot(noise, means, c='steelblue', marker='o')
ax.fill_between(noise, means-stds,means+stds,alpha=.1, color='steelblue')
ax.set_xlabel('noise [px]')
ax.set_ylabel('error [px]')

plt.savefig('output/noise_results_06.png', dpi=300)

In [ ]:
# TODO ADD 0!!!
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

grouped = noise_results_0029.groupby('n_std').agg(
    error_mean=('error', lambda x: np.mean(x).round(2)),
    error_std=('error', lambda x: np.std(x).round(2))
).reset_index()

means = grouped['error_mean'].values
stds = grouped['error_std'].values
noise = grouped['n_std'].values

# Add zero experiment
means = np.insert(means, 0, 3.83, axis=0)
stds = np.insert(stds, 0, 0, axis=0)
noise = np.insert(noise, 0, 0, axis=0)

ax.plot(noise, means, c='steelblue', marker='o')
ax.fill_between(noise, means-stds,means+stds,alpha=.1, color='steelblue')
ax.set_xlabel('noise [px]')
ax.set_ylabel('error [px]')

plt.savefig('output/noise_results_02.png', dpi=300)

In [ ]:
noise_results = noise_results_0026
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
noise_results.boxplot(
    column='error',
    by='n_std',
    showfliers=True,
    vert=False,
    patch_artist=True,
    boxprops=dict(facecolor='lightgray', lw=1),
    medianprops=dict(color='black', lw=2),
    whiskerprops = dict(color = "black", lw=1),
    widths=0.5,
    ax=ax
)
ax.set_title('')
fig.suptitle('')
ax.set_ylabel('standard deviation')
ax.set_xlabel('error')
ax.axvline(6.95, lw=3, ls='dashed', color='black')
plt.savefig('output/noised_experiment_results.png', dpi=300)
plt.show()

In [ ]:
# HOW CAN THE PIXEL DENSITY BE BIGGER IN CASE n_STD = 50?? THEY SHOULD BE ALWAYS THE SAME ... 
# Pixel densities will be slightly off, depending on how far you move the reference point,
# your predicted validation points into the perspective view will be off!
# You should use a "perfect" Homography for this; so the 6.95 basic homography!


# noise_results.loc[noise_results['n_std'] == 10].plot.scatter(
noise_results.loc[noise_results['n_std'] == 10].plot.scatter(
    x='pixel_density',
    y='error',
    # c='n_std',
    s= 200, 
    # colormap='viridis',
    # alpha=0.4,
    figsize=(15, 10)
)
plt.show()

In [ ]:
# I Think some kind of barchart should be better

fig, ax = plt.subplots(1, 1, figsize=(12, 10))
noise_results.loc[noise_results['n_std'] == 50].boxplot(
    column='error',
    by='pixel_density',
    showfliers=True,
    vert=False,
    patch_artist=True,
    boxprops=dict(facecolor='lightgray', lw=1),
    medianprops=dict(color='black', lw=2),
    whiskerprops = dict(color = "black", lw=1),
    widths=0.5,
    ax=ax
)
# ax.axvline(6.95, lw=3)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
noise_results.loc[noise_results['n_std'] == 5].boxplot(
    column='error',
    by='pixel_density',
    showfliers=True,
    vert=False,
    patch_artist=True,
    boxprops=dict(facecolor='lightgray', lw=1),
    medianprops=dict(color='black', lw=2),
    whiskerprops = dict(color = "black", lw=1),
    widths=0.5,
    ax=ax
)
# ax.axvline(6.95, lw=3)
plt.show()

## 2. Outliers

In [ ]:
def conduct_experiment_outlier(pv_img, tv_img, basic_homography, plot=True, print_errors=True, used_ref_points=None, n_mean=0, n_std=100):
    # Load Data
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)
    
    if used_ref_points:
        reference_pts_pv = filter_points(reference_pts_pv, used_ref_points)
        reference_pts_tv = filter_points(reference_pts_tv, used_ref_points)
        
    # Transform Points to Homogeneous Numpy arrays
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)

    # Add Noise
    reference_pts_pv_arr_noised = add_outlier(reference_pts_pv_arr, 2, 50)
    reference_pts_tv_arr_noised = add_outlier(reference_pts_tv_arr, 2, 50)
    reference_pts_pv_arr_noised = add_outlier(reference_pts_pv_arr_noised, 3, 50)
    reference_pts_tv_arr_noised = add_outlier(reference_pts_tv_arr_noised, 3, 50)
    
    # Calculate Homography
    h_r, _ = cv2.findHomography(
        reference_pts_pv_arr_noised,
        reference_pts_tv_arr_noised,
        method = cv2.RANSAC,
        # method = 0,
    )
    h_r_inv = np.linalg.inv(h_r)
    
    h_ls, _ = cv2.findHomography(
        reference_pts_pv_arr_noised,
        reference_pts_tv_arr_noised,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_ls_inv = np.linalg.inv(h_ls)

    reference_results = []
    validation_results = []
    names = ['ls', 'r']
    i = 0
    for h, h_inv in ((h_ls, h_ls_inv), (h_r, h_r_inv)):
        # Get Pixel Densities
        reference_densities = get_densities(reference_pts_tv_arr, h_inv)
        # The densities should be calculated "perfectly" for the validation point, 
        # hence we should use a good homography for this, 
        # hence, we choose the baseline homography
        validation_densities = get_densities(validation_pts_tv_arr, np.linalg.inv(basic_homography)) # important trick!
        
        # Predict Points
        predicted_reference_pts_tv = predict(reference_pts_pv_arr_noised, h)
        predicted_validation_pts_tv = predict(validation_pts_pv_arr, h)
        
        # Errors
        reference_errors = calculate_errors(predicted_reference_pts_tv, reference_pts_tv_arr)
        validation_errors = calculate_errors(predicted_validation_pts_tv, validation_pts_tv_arr)
    
        # Combine Results in Dictionary
        reference_result_dict = get_result_dict_noised(
            reference_pts_pv, 
            reference_pts_tv_arr, 
            predicted_reference_pts_tv, 
            reference_errors, 
            reference_densities, 
            reference_pts_pv_arr_noised, 
            reference_pts_tv_arr_noised
        )
        validation_result_dict = get_result_dict(validation_pts_pv, validation_pts_tv_arr, predicted_validation_pts_tv, validation_errors, validation_densities, reference_pts_pv_arr)
         
        # Print Errors
        if print_errors:
            print(f'Reference  Error: {np.mean(reference_errors) :.2f} PX!')
            print(f'Reference  Error: {np.mean(reference_errors) / SCALING_FACTOR * 100 :.2f} CM!')
            print('---')
            print(f'Validation Error: {np.mean(validation_errors):.2f} PX!')
            print(f'Validation Error: {np.mean(validation_errors) / SCALING_FACTOR * 100 :.2f} CM!')
         
        # Plots
        if plot:
            plot_setup_noised(img_tv, img_pv, reference_result_dict, validation_result_dict)
            plot_predictions(img_tv, img_pv, reference_result_dict, validation_result_dict, names[i])
            i+=1
        reference_results.append(reference_result_dict), validation_results.append(validation_result_dict)
    plt.show()
    return reference_results, validation_results
baseline_val, baseline_ref, baseline_h = conduct_experiment('IMG_01', 'IMG_00', plot=False, return_homography=True)
reference_results, validation_results = conduct_experiment_outlier('IMG_01', 'IMG_00', baseline_h, plot=True) 

In [ ]:
_ = conduct_experiment('IMG_01', 'IMG_00', plot=True)